In [1]:
import pathlib
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt

In [3]:
AMD_CSV_PATH = pathlib.Path('../../docs/2017_09_pact/data/case-study-a/cgo17-amd.csv')
NVIDIA_CSV_PATH = pathlib.Path('../../docs/2017_09_pact/data/case-study-a/cgo17-nvidia.csv')
KERNELS_TAR_PATH = pathlib.Path('../../docs/2017_09_pact/data/case-study-a/kernels.tar.bz2')

assert AMD_CSV_PATH.is_file()
assert NVIDIA_CSV_PATH.is_file()
assert KERNELS_TAR_PATH.is_file()

amd_df = pd.read_csv(AMD_CSV_PATH)
nvidia_df = pd.read_csv(NVIDIA_CSV_PATH)
amd_df.drop('Unnamed: 0', axis='columns', inplace=True)
nvidia_df.drop('Unnamed: 0', axis='columns', inplace=True)

In [12]:
import tarfile

srcs = {}
with tarfile.open(KERNELS_TAR_PATH, "r:bz2") as tar:
    for member in [member for member in tar.getmembers() if member.isfile()]:
        assert member.name.startswith('kernels/')
        assert member.name.endswith('.cl')
        benchmark = member.name[len('kernels/'):]
        srcs[benchmark] = tar.extractfile(member).read()

print(len(srcs))

256


In [19]:
df = amd_df.join(nvidia_df, lsuffix='_amd', rsuffix='_nvidia')
assert all(x == y for x, y in df[['benchmark_amd', 'benchmark_nvidia']].values)
assert all(x == y for x, y in df[['dataset_amd', 'dataset_nvidia']].values)

df['program:opencl_kernel_name'] = [b.split('-')[-1] for b in df['benchmark_amd']]
df['program:benchmark_name'] = [b.split('-')[-2] for b in df['benchmark_amd']]
df['program:benchmark_suite_name'] = ['-'.join(b.split('-')[:-2]) for b in df['benchmark_amd']]

df.rename({
    'dataset_amd': 'data:dataset_name',
    'benchmark_amd': 'benchmark',
    'wgsize_amd': 'wgsize'
}, axis='columns', inplace=True)
df.drop([
    'benchmark_nvidia',
    'dataset_nvidia',
], axis='columns', inplace=True)
df.columns.values

array(['benchmark', 'data:dataset_name', 'comp_amd', 'rational_amd',
       'mem_amd', 'localmem_amd', 'coalesced_amd', 'atomic_amd',
       'transfer_amd', 'wgsize', 'oracle_amd', 'runtime_cpu_amd',
       'runtime_gpu_amd', 'src_amd', 'seq_amd', 'comp_nvidia',
       'rational_nvidia', 'mem_nvidia', 'localmem_nvidia',
       'coalesced_nvidia', 'atomic_nvidia', 'transfer_nvidia',
       'wgsize_nvidia', 'oracle_nvidia', 'runtime_cpu_nvidia',
       'runtime_gpu_nvidia', 'src_nvidia', 'seq_nvidia',
       'program:opencl_kernel_name', 'program:benchmark_name',
       'program:benchmark_suite_name'], dtype=object)

In [29]:
f = df.groupby('benchmark').min()
print(len(f))
f

256


,data:dataset_name,comp_amd,rational_amd,mem_amd,localmem_amd,coalesced_amd,atomic_amd,transfer_amd,wgsize,oracle_amd,...,transfer_nvidia,wgsize_nvidia,oracle_nvidia,runtime_cpu_nvidia,runtime_gpu_nvidia,src_nvidia,seq_nvidia,program:opencl_kernel_name,program:benchmark_name,program:benchmark_suite_name
benchmark,,,,,,,,,,,,,,,,,,,,,
amd-app-sdk-3.0-BinomialOption-binomial_options,default,98,8,13,11,2,0,2048,255,GPU,...,2048,255,GPU,3.291073,0.152038,"__kernel void A(int a, const __global float4* ...","[129 129 129 ..., 0 127 0]",binomial_options,BinomialOption,amd-app-sdk-3.0
amd-app-sdk-3.0-BitonicSort-bitonicSort,default,12,2,6,0,6,0,131072,256,CPU,...,131072,256,GPU,0.121940,0.057834,"__kernel void A(__global uint* a, const uint b...","[129 129 129 ..., 0 127 0]",bitonicSort,BitonicSort,amd-app-sdk-3.0
amd-app-sdk-3.0-BlackScholes-blackScholes,default,220,4,3,0,0,0,3145728,256,GPU,...,3145728,256,GPU,6.076052,1.997728,"__kernel void A(const __global float4* a, int ...","[129 129 129 ..., 26 0 127]",blackScholes,BlackScholes,amd-app-sdk-3.0
amd-app-sdk-3.0-FastWalshTransform-fastWalshTransform,default,8,0,4,0,4,0,4096,256,CPU,...,4096,256,GPU,0.130003,0.062024,"__kernel void A(__global float* a, __const int...","[129 129 129 ..., 0 127 0]",fastWalshTransform,FastWalshTransform,amd-app-sdk-3.0
amd-app-sdk-3.0-FloydWarshall-floydWarshallPass,default,11,1,5,0,0,0,524288,256,CPU,...,524288,256,CPU,0.283447,0.604686,"__kernel void A(__global uint* a, __global uin...","[129 129 129 ..., 127 0 127]",floydWarshallPass,FloydWarshall,amd-app-sdk-3.0
amd-app-sdk-3.0-MatrixMultiplication-mmmKernel,default,163,1,12,0,0,0,49152,64,CPU,...,49152,64,GPU,0.277644,0.124211,"__kernel void A(__global float4* a, __global f...","[ 1 9 1 ..., 0 127 0]",mmmKernel,MatrixMultiplication,amd-app-sdk-3.0
amd-app-sdk-3.0-MatrixTranspose-matrixTranspose,default,42,0,16,8,2,0,32768,256,CPU,...,32768,256,GPU,0.156639,0.087846,"__kernel void A(__global float4* a, __global f...","[129 129 129 ..., 0 127 0]",matrixTranspose,MatrixTranspose,amd-app-sdk-3.0
amd-app-sdk-3.0-PrefixSum-group_prefixSum,default,36,10,12,8,2,0,4096,640,CPU,...,4096,1024,GPU,0.484234,0.049376,"__kernel void A(__global float* a, __global fl...","[129 129 129 ..., 0 127 0]",group_prefixSum,PrefixSum,amd-app-sdk-3.0
amd-app-sdk-3.0-Reduction-reduce,default,6,3,7,4,2,0,8208,256,CPU,...,8208,256,GPU,0.161025,0.070912,"__kernel void A(__global uint4* a, __global ui...","[129 129 129 ..., 0 127 0]",reduce,Reduction,amd-app-sdk-3.0


In [9]:
src_df = runtimes_df[runtimes_df.cf == 1][['kernel', 'src']]
src_df.rename({
    'kernel': 'program:opencl_kernel_name',
    'src': 'program:opencl_kernel_src',
}, axis='columns', inplace=True)
src_df.set_index('program:opencl_kernel_name', inplace=True)
src_df.sort_index(inplace=True)
# src_df

NameError: name 'runtimes_df' is not defined

In [36]:
for level in list(level.name for level in df.set_index(['benchmark', 'comp_amd']).index.levels):
    print(level.name)

benchmark
comp_amd
